In [0]:
import pandas as pd

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
bank_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bank.csv')

In [20]:
bank_df.shape

(10000, 14)

In [21]:
bank_df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [22]:
bank_df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [23]:
bank_df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [24]:
list1 = bank_df['RowNumber'].value_counts() > 1
list2 = bank_df['CustomerId'].value_counts() > 1
print(list1.unique())
print(list2.unique())

[False]
[False]


In [0]:
#Row number, CustomerId looks disctinct for each record and is fairly not very useful for our 
#analysis in classifying the customers

#Surname column is not very useful in our analysis and hence we can skip Surname as well

In [0]:
bank_df.drop({'RowNumber','CustomerId', 'Surname'}, axis=1, inplace=True)

In [27]:
bank_df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [0]:
X = bank_df.drop('Exited', axis = 1)
y = bank_df['Exited']

In [29]:
X.shape

(10000, 10)

In [30]:
y.shape

(10000,)

In [31]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
labelEncoder = LabelEncoder()
X['Geography'] = labelEncoder.fit_transform(X['Geography'])
X['Gender'] = labelEncoder.fit_transform(X['Gender'])

In [34]:
X.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10
5,645,2,1,44,8,113755.78,2,1,0,149756.71
6,822,0,1,50,7,0.00,2,1,1,10062.80
7,376,1,0,29,4,115046.74,4,1,0,119346.88
8,501,0,1,44,4,142051.07,2,0,1,74940.50
9,684,0,1,27,2,134603.88,1,1,1,71725.73


In [0]:
from scipy.stats import zscore

In [0]:
X = X.apply(zscore)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state = 40)

In [39]:
train_X.shape

(7500, 10)

In [40]:
train_y.shape

(7500,)

In [41]:
train_X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
8925,0.325625,0.306591,0.912419,-0.660018,-1.041760,0.459625,0.807737,0.646092,0.970243,1.114291
809,0.066956,-0.901886,-1.095988,-0.183251,0.341352,0.310280,-0.911583,0.646092,-1.030670,0.384784
5903,-0.833213,-0.901886,-1.095988,-0.946079,-0.350204,1.226942,-0.911583,0.646092,0.970243,0.543812
9107,0.046263,1.515067,0.912419,-0.087897,1.378686,-1.225848,-0.911583,-1.547768,0.970243,-0.166933
4941,-0.108939,1.515067,0.912419,0.388871,1.378686,1.538478,-0.911583,0.646092,-1.030670,1.582298


In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [43]:
tf.__version__

'2.1.0-rc1'

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [0]:
model = Sequential()
model.add(Dense(6, activation = 'relu', input_shape = (train_X.shape[1], )))
model.add(Dropout(rate=0.1)) 
model.add(Dense(6, activation = 'relu')) 
model.add(Dropout(rate=0.1))
model.add(Dense(1, activation = 'sigmoid'))

In [68]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 6)                 66        
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 6)                 42        
_________________________________________________________________
dropout_7 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 7         
Total params: 115
Trainable params: 115
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy']) #As our dependent variable is Exited with 0 and 1 values we can use binary loss function

In [70]:
history = model.fit(train_X, train_y, batch_size=50, epochs=20, validation_split=0.1, verbose=2)

Train on 6750 samples, validate on 750 samples
Epoch 1/20
6750/6750 - 1s - loss: 0.6402 - accuracy: 0.6988 - val_loss: 0.5395 - val_accuracy: 0.8213
Epoch 2/20
6750/6750 - 0s - loss: 0.5514 - accuracy: 0.7867 - val_loss: 0.4748 - val_accuracy: 0.8253
Epoch 3/20
6750/6750 - 0s - loss: 0.5184 - accuracy: 0.7893 - val_loss: 0.4426 - val_accuracy: 0.8253
Epoch 4/20
6750/6750 - 0s - loss: 0.4970 - accuracy: 0.7886 - val_loss: 0.4215 - val_accuracy: 0.8253
Epoch 5/20
6750/6750 - 0s - loss: 0.4792 - accuracy: 0.7910 - val_loss: 0.4073 - val_accuracy: 0.8280
Epoch 6/20
6750/6750 - 0s - loss: 0.4670 - accuracy: 0.7947 - val_loss: 0.4000 - val_accuracy: 0.8267
Epoch 7/20
6750/6750 - 0s - loss: 0.4650 - accuracy: 0.7939 - val_loss: 0.3966 - val_accuracy: 0.8307
Epoch 8/20
6750/6750 - 0s - loss: 0.4547 - accuracy: 0.7959 - val_loss: 0.3926 - val_accuracy: 0.8307
Epoch 9/20
6750/6750 - 0s - loss: 0.4525 - accuracy: 0.7959 - val_loss: 0.3913 - val_accuracy: 0.8333
Epoch 10/20
6750/6750 - 0s - loss: 

In [72]:
y_pred = model.predict(test_X)
print(y_pred[:5])

[[0.23702152]
 [0.33543906]
 [0.02135882]
 [0.15465595]
 [0.05900595]]


In [73]:
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [74]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, y_pred)
print(cm)

[[1976   35]
 [ 435   54]]


In [0]:
# Optimizing the parameters
model2 = Sequential()
model2.add(Dense(20, activation = 'relu', input_shape = (train_X.shape[1], )))
model2.add(Dropout(rate=0.1)) 
model2.add(Dense(10, activation = 'relu')) 
model2.add(Dropout(rate=0.1))
model2.add(Dense(5, activation = 'relu'))
model2.add(Dropout(rate=0.1))
model2.add(Dense(1, activation = 'sigmoid'))

In [0]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [95]:
history2 = model2.fit(train_X, train_y, batch_size=32, epochs=30, validation_split=0.1, verbose=2)

Train on 6750 samples, validate on 750 samples
Epoch 1/30
6750/6750 - 1s - loss: 0.5792 - accuracy: 0.7600 - val_loss: 0.4520 - val_accuracy: 0.8280
Epoch 2/30
6750/6750 - 0s - loss: 0.4892 - accuracy: 0.7941 - val_loss: 0.4016 - val_accuracy: 0.8360
Epoch 3/30
6750/6750 - 0s - loss: 0.4579 - accuracy: 0.8015 - val_loss: 0.3866 - val_accuracy: 0.8387
Epoch 4/30
6750/6750 - 0s - loss: 0.4494 - accuracy: 0.8046 - val_loss: 0.3814 - val_accuracy: 0.8413
Epoch 5/30
6750/6750 - 0s - loss: 0.4348 - accuracy: 0.8049 - val_loss: 0.3756 - val_accuracy: 0.8507
Epoch 6/30
6750/6750 - 0s - loss: 0.4311 - accuracy: 0.8062 - val_loss: 0.3790 - val_accuracy: 0.8453
Epoch 7/30
6750/6750 - 0s - loss: 0.4237 - accuracy: 0.8043 - val_loss: 0.3762 - val_accuracy: 0.8467
Epoch 8/30
6750/6750 - 0s - loss: 0.4222 - accuracy: 0.8132 - val_loss: 0.3672 - val_accuracy: 0.8480
Epoch 9/30
6750/6750 - 0s - loss: 0.4145 - accuracy: 0.8213 - val_loss: 0.3579 - val_accuracy: 0.8600
Epoch 10/30
6750/6750 - 0s - loss: 

In [96]:
y_pred2 = model2.predict(test_X)
print(y_pred2[:5])

[[0.21926291]
 [0.18745962]
 [0.01525299]
 [0.18683927]
 [0.03624965]]


In [97]:
y_pred2 = (y_pred2 > 0.5).astype(int)
print(y_pred2[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [98]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, y_pred2)
print(cm)

[[1944   67]
 [ 273  216]]
